In [20]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import json
from glob import glob
import emoji
import re

In [2]:
data_pathes: list[str] = glob("dataest/toxic/[!twitter_crawl]*")


In [3]:
series = []
for path in data_pathes:
    series.append(
        pd.DataFrame.from_dict(pd.read_json(path)['messages'].tolist())
        )

df = pd.concat(series, ignore_index=True)

In [5]:
def preprocess(text: str):
    chars = set(text)
    removed_emoji = list(filter(lambda x: not emoji.is_emoji(x), chars))
    if len(removed_emoji) > 5:
        return text
    return None

In [6]:
preprocess(df.iloc[6832]['text'])

'زِندآنی شُدیم توکِشوَری کِع شُعآرِش آزآدیِع😹💋🏹🖇🌵'

In [10]:
cleaned_data = []
for i in df['text']:
    if type(i) == str:
        if i != "":
            cleaned_data.append({"text": preprocess(i), "toxic": 1})

In [11]:
text_df = pd.DataFrame.from_dict(cleaned_data)

In [15]:
text_df.dropna(inplace=True)

In [16]:
text_df.to_csv("telegram.csv", index=False)

# Twitter

In [31]:
twitts_df = pd.read_csv("dataest/not toxic/tweets-politicians.csv")

In [30]:
twitts_df.iloc[84207]['tweet']

'در اهمیّت حرکت به سوی #شایسته_سالاری https://twitter.com/abdollahram/status/853967665270063108\xa0…'

In [32]:
def username_cleaner(text):
    removed_username = re.sub('@[^\s]+','',text)
    return re.sub(r'http\S+', '', removed_username)

In [40]:
cleaned_twitter = twitts_df['text'].map(username_cleaner)

In [42]:
cleaned_twitter = pd.DataFrame(cleaned_twitter, columns=['text'])

In [43]:
cleaned_twitter['toxic'] = [1] * len(cleaned_twitter) 

In [45]:
cleaned_twitter.to_csv("twitter-toxic.csv", index=False)

# tweet politician

In [51]:
twitts_df = pd.read_csv("dataest/not toxic/tweets-politicians.csv")

/tmp/ipykernel_4509/3675416897.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  twitts_df = pd.read_csv("dataest/not toxic/tweets-politicians.csv")


In [49]:
twitts_df.keys()

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'mentions', 'urls',
       'photos', 'replies_count', 'retweets_count', 'likes_count', 'hashtags',
       'cashtags', 'link', 'retweet', 'quote_url', 'video', 'near', 'geo',
       'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to',
       'retweet_date', 'translate', 'trans_src', 'trans_dest'],
      dtype='object')

In [62]:
cleaned_twitter = twitts_df['tweet'].map(username_cleaner)
cleaned_twitter = pd.DataFrame(cleaned_twitter, columns=['tweet']).rename(columns= {'tweet': 'text'})
# cleaned_twitter = pd.DataFrame(cleaned_twitter, columns=['text'])
cleaned_twitter['toxic'] = [0] * len(cleaned_twitter) 
cleaned_twitter.to_csv("twitter-notoxic.csv", index=False)

In [56]:
cleaned_twitter = twitts_df['tweet'].map(username_cleaner)

In [60]:
pd.DataFrame(cleaned_twitter, columns=['tweet']).rename(columns= {'tweet': 'text'})

,text
0,نسل جوان ما بحمدالله الگوهای درخشان و برجسته و...
1,نسل جوانی که در مسیر #علم_نافع، برای پیشرفت کش...
2,کودک و نوجوان و جوان در کشور اسلامی می‌آموزد ک...
3,در مدرسه نبوت،نسلهای بشر با کتاب وحکمت،آموزش و...
4,این سخن امام خمینی که معلمی را شغل انبیا دانست...
...,...
84205,این گفتگو در جمع دوستان بدرقه کننده و پیش از ع...
84206,نقش #احمدی_نژاد در تعطیل شدن دانشگاه‌ها به بها...
84207,در اهمیّت حرکت به سوی #شایسته_سالاری …
84208,تأکید 9چهره اصلاحات بر شایسته سالاری\nنامه به ...


In [67]:
series = []
for path in ['telegram.csv', "twitter-notoxic.csv", "twitter-toxic.csv"]:
    series.append(
        pd.read_csv(path)
        )

df = pd.concat(series, ignore_index=True)

In [73]:
df.dropna(inplace=True)

In [76]:
df.toxic = df.toxic.astype(int)

In [77]:
df.to_csv("dataset.csv")